In [1]:
import pandas as pd
import glob
import math, statistics
import csv
import re

path_on= r"C:/Users/erick/OneDrive/Desktop/MT-FBK-internship/on-off/on_eng/heroes*.csv"
path_off= r"C:/Users/erick/OneDrive/Desktop/MT-FBK-internship/on-off/off_eng/heroes*.csv"
path_mix= r"C:/Users/erick/OneDrive/Desktop/MT-FBK-internship/on-off/mixed_eng/heroes*.csv"

path_on_sp = r"C:/Users/erick/OneDrive/Desktop/MT-FBK-internship/on-off/on_spa/heroes*.csv"
path_off_sp = r"C:/Users/erick/OneDrive/Desktop/MT-FBK-internship/on-off/off_spa/heroes*.csv"
path_mix_sp = r"C:/Users/erick/OneDrive/Desktop/MT-FBK-internship/on-off/mixed_spa/heroes*.csv"

# dat function returns all the frequency and intensity values for each word for each segment per path

def dat_function(path):
    freq_list = []
    int_list = []

    for fname in glob.glob(path):
        df = pd.read_csv(fname, delimiter='|')
        for index, row in df.iterrows():
            if abs(row.f0_mean) > 0:
                freq_list.append(row.f0_mean)
            if abs(row.i0_mean) > 0:
                int_list.append(row.i0_mean)
            else:
                pass
    return freq_list, int_list


In [2]:
on_res = dat_function(path_on)
off_res = dat_function(path_off)
mix_res = dat_function(path_mix)
on_sp_res = dat_function(path_on_sp)
off_sp_res = dat_function(path_off_sp)
mix_sp_res = dat_function(path_mix_sp)

pairs = [(on_res, on_sp_res), (off_res, off_sp_res), (mix_res, mix_sp_res)]

all_eng = (on_res[0] + off_res[0] + mix_res[0], on_res[1] + off_res[1] + mix_res[1])
all_spa = (on_sp_res[0] + off_sp_res[0] + mix_sp_res[0], on_sp_res[1] + off_sp_res[1] + mix_sp_res[1])

In [3]:
print('FREQ\n')
print('english on ', statistics.stdev(on_res[0]), statistics.stdev(on_res[1]))
print('english off ', statistics.stdev(off_res[0]), statistics.stdev(off_res[1]))
print('english mixed ', statistics.stdev(mix_res[0]), statistics.stdev(mix_res[1]))
print('\nINTENSITY\n')
print('spanish on ', statistics.stdev(on_sp_res[0]), statistics.stdev(on_sp_res[1]))
print('spanish off ', statistics.stdev(off_sp_res[0]), statistics.stdev(off_sp_res[1]))
print('spanish mixed ', statistics.stdev(mix_sp_res[0]), statistics.stdev(mix_sp_res[1]))

#absolute values
freq_abs_mean = statistics.mean(all_eng[0])
int_abs_mean = statistics.mean(all_eng[1])

print('mean freq ',freq_abs_mean)
print('mean int ',int_abs_mean)

freq_abs_stdev = statistics.stdev(all_eng[0])
int_abs_stdev = statistics.stdev(all_eng[1])

#screen-dep values
freq_on_mean = statistics.mean(on_res[0])
freq_off_mean = statistics.mean(off_res[0])
freq_mix_mean = statistics.mean(mix_res[0])

freq_on_stdev = statistics.stdev(on_res[0])
freq_off_stdev = statistics.stdev(off_res[0])
freq_mix_stdev = statistics.stdev(mix_res[0])

int_on_mean = statistics.mean(on_res[1])
int_off_mean = statistics.mean(off_res[1])
int_mix_mean = statistics.mean(mix_res[1])

int_on_stdev = statistics.stdev(on_res[1])
int_off_stdev = statistics.stdev(off_res[1])
int_mix_stdev = statistics.stdev(mix_res[1])

FREQ

english on  8.85821993820414 1.263712124324298
english off  9.603564516478704 1.206925483779559
english mixed  8.977168297816956 1.237321688667108

INTENSITY

spanish on  7.736198155872236 1.1587411549034985
spanish off  9.290136948045738 1.1542584215810092
spanish mixed  7.730712013092112 1.1277217029652018
mean freq  -2.29208069603407
mean int  -0.045192339592417904


In [4]:
import pandas as pd
import glob
import math, statistics
import csv
import re

path_on= r"C:/Users/erick/OneDrive/Desktop/MT-FBK-internship/on-off/on_eng/heroes*.csv"
path_on_sp = r"C:/Users/erick/OneDrive/Desktop/MT-FBK-internship/on-off/on_spa/heroes*.csv"

path_off= r"C:/Users/erick/OneDrive/Desktop/MT-FBK-internship/on-off/off_eng/heroes*.csv"
path_off_sp = r"C:/Users/erick/OneDrive/Desktop/MT-FBK-internship/on-off/off_spa/heroes*.csv"

path_mix= r"C:/Users/erick/OneDrive/Desktop/MT-FBK-internship/on-off/mixed_eng/heroes*.csv"
path_mix_sp = r"C:/Users/erick/OneDrive/Desktop/MT-FBK-internship/on-off/mixed_spa/heroes*.csv"

# this func. gets the word, (shortened)id, f0_mean and i0_mean for each word for each segment file in the path
def pre_peak_finder(path):
    final_list = []
    for fname in glob.glob(path):
        temp_l = ()
        temp_seg = []
        t_df = pd.read_csv(fname, delimiter='|')
        for index, row in t_df.iterrows():
            id_i = str(row.id[:13])+str(row.id[24:])
            temp_l = (row.word, id_i, row.f0_mean, row.i0_mean)
            temp_seg.append(temp_l)
            temp_l = ()
        final_list.append(temp_seg)
        temp_seg = []
    return final_list

In [5]:
# calling pre_peak_finder func. Use these pairs as input for peak_finder func.

a = pre_peak_finder(path_on)
b = pre_peak_finder(path_on_sp)

m = pre_peak_finder(path_off)
n = pre_peak_finder(path_off_sp)

x =  pre_peak_finder(path_mix)
y =  pre_peak_finder(path_mix_sp)

In [6]:
def segment_peak_finder(f):
    freq_peaks = []
    int_peaks = []
    ids_freq = []
    ids_int = []
    
    for seg in f:
        for w in seg:
            # frequency peaks
            if w[2] >= (freq_abs_mean + 2*freq_abs_stdev) or w[2] <= (freq_abs_mean - 2*freq_abs_stdev):
                if seg not in freq_peaks:
                    freq_peaks.append(seg)
                else:
                    pass
                if w[1] not in ids_freq:
                    ids_freq.append([w[1], w[0], w[2], w[3]])
                else:
                    pass
            # intensity peaks
            if w[3] >= (int_abs_mean + 2*int_abs_stdev) or w[3] <= (int_abs_mean - 2*int_abs_stdev):
                int_peaks.append(seg)
                if w[1] not in ids_int:
                    ids_int.append([w[1], w[0], w[2], w[3]])
                else:
                    pass
            else:
                pass
            
    return freq_peaks, int_peaks, ids_freq, ids_int
            

In [7]:
segs_en_on = segment_peak_finder(a)
segs_spa_on = segment_peak_finder(b)

segs_en_off = segment_peak_finder(m)
segs_spa_off = segment_peak_finder(n)

segs_en_mix = segment_peak_finder(x)
segs_spa_mix = segment_peak_finder(y)

import pandas as pd

def df_maker(data):
    my_df_freq = pd.DataFrame({'segment': [i[0].split('.word')[0] for i in data[2]], 
                          'word_id': [i[0].split('.word')[1] for i in data[2]], 
                          'word': [i[1] for i in data[2]], 'frequency': [i[2] for i in data[2]],
                         'intensity': [i[3] for i in data[2]]})
    
    my_df_int = pd.DataFrame({'segment': [i[0].split('.word')[0] for i in data[3]], 
                          'word_id': [i[0].split('.word')[1] for i in data[3]], 
                          'word': [i[1] for i in data[3]], 'frequency': [i[2] for i in data[3]],
                         'intensity': [i[3] for i in data[3]]})
    
    return my_df_freq, my_df_int

df_maker(segs_spa_on)

(                     segment word_id         word  frequency  intensity
 0    heroes_s2_10_.segment24       4     familiar    19.9629    -0.0484
 1    heroes_s2_10_.segment38       1         solo    27.1037     0.0121
 2    heroes_s2_10_.segment38       2          así    27.4934     0.0121
 3    heroes_s2_10_.segment51       3     ocurrido    23.1611    -1.1494
 4    heroes_s2_10_.segment67       1     mohinder   -23.9163    -0.6436
 ..                       ...     ...          ...        ...        ...
 924  heroes_s3_9_ssegment313       6        hasta   -25.2302    -1.6368
 925  heroes_s3_9_ssegment313       7          hoy   -23.2249     0.0674
 926  heroes_s3_9_ssegment314       1         creo   -23.7746    -0.2052
 927  heroes_s3_9_ssegment314       4           el   -36.9292    -2.5563
 928  heroes_s3_9_ssegment314       5  catalizador   -26.8197    -1.0499
 
 [929 rows x 5 columns],
                      segment word_id       word  frequency  intensity
 0    heroes_s2_10_.segmen

In [8]:
# obtaining common elements between paired datasets

def commonalities(a, b):
        
    common_freq_ = set.intersection(set(a[0].segment), set(b[0].segment))
    common_int_ = set.intersection(set(a[1].segment), set(b[1].segment))
    
    return common_freq_, common_int_

print('on/freq ',len(commonalities(df_maker(segs_en_on), df_maker(segs_spa_on))[0]))
print('on/int ',len(commonalities(df_maker(segs_en_on), df_maker(segs_spa_on))[1]))

print('off/freq ',len(commonalities(df_maker(segs_en_off), df_maker(segs_spa_off))[0]))
print('off/int ',len(commonalities(df_maker(segs_en_off), df_maker(segs_spa_off))[1]))

print('mix/freq ',len(commonalities(df_maker(segs_en_mix), df_maker(segs_spa_mix))[0]))
print('mix/int ',len(commonalities(df_maker(segs_en_mix), df_maker(segs_spa_mix))[1]))


on/freq  97
on/int  66
off/freq  46
off/int  19
mix/freq  63
mix/int  56


The names of some English files didn't match those of their Spanish counterparts. The issue was fixed by editing the csv files manually. The files in the code below contain the corrected names.

In [9]:
df_maker(segs_spa_on)[1].to_csv('segments_int_spanish_on_test.csv')
df_maker(segs_spa_off)[1].to_csv('segments_int_spanish_off_test.csv')
df_maker(segs_spa_mix)[1].to_csv('segments_int_spanish_mix_test.csv')
df_maker(segs_en_on)[1].to_csv('segments_int_english_on_test.csv')
df_maker(segs_en_off)[1].to_csv('segments_int_english_off_test.csv')
df_maker(segs_en_mix)[1].to_csv('segments_int_english_mix_test.csv')

In [10]:
df_sp_on_f = pd.read_csv('C:/Users/erick/OneDrive/Desktop/pros_results/new_peaks/segments_spanish_on_test.csv')
df_sp_off_f = pd.read_csv('C:/Users/erick/OneDrive/Desktop/pros_results/new_peaks/segments_spanish_off_test.csv')
df_sp_mix_f = pd.read_csv('C:/Users/erick/OneDrive/Desktop/pros_results/new_peaks/segments_spanish_mix_test.csv')
df_en_on_f = pd.read_csv('C:/Users/erick/OneDrive/Desktop/pros_results/new_peaks/segments_english_on_test.csv')
df_en_off_f = pd.read_csv('C:/Users/erick/OneDrive/Desktop/pros_results/new_peaks/segments_english_off_test.csv')
df_en_mix_f = pd.read_csv('C:/Users/erick/OneDrive/Desktop/pros_results/new_peaks/segments_english_mix_test.csv')

df_sp_on_i = pd.read_csv('C:/Users/erick/OneDrive/Desktop/pros_results/new_peaks/segments_int_spanish_on_test.csv')
df_sp_off_i = pd.read_csv('C:/Users/erick/OneDrive/Desktop/pros_results/new_peaks/segments_int_spanish_off_test.csv')
df_sp_mix_i = pd.read_csv('C:/Users/erick/OneDrive/Desktop/pros_results/new_peaks/segments_int_spanish_mix_test.csv')
df_en_on_i = pd.read_csv('C:/Users/erick/OneDrive/Desktop/pros_results/new_peaks/segments_int_english_on_test.csv')
df_en_off_i = pd.read_csv('C:/Users/erick/OneDrive/Desktop/pros_results/new_peaks/segments_int_english_off_test.csv')
df_en_mix_i = pd.read_csv('C:/Users/erick/OneDrive/Desktop/pros_results/new_peaks/segments_int_english_mix_test.csv')

In [12]:
# Variables containing the set of segments with concurrent peaks

common_on_freq = set.intersection(set(df_en_on_f.segment), set(df_sp_on_f.segment))
common_off_freq = set.intersection(set(df_en_off_f.segment), set(df_sp_off_f.segment))
common_mix_freq = set.intersection(set(df_en_mix_f.segment), set(df_sp_mix_f.segment))

common_on_int = set.intersection(set(df_en_on_i.segment), set(df_sp_on_i.segment))
common_off_int = set.intersection(set(df_en_off_i.segment), set(df_sp_off_i.segment))
common_mix_int = set.intersection(set(df_en_mix_i.segment), set(df_sp_mix_i.segment))

In [ ]:
print('freq on: ', len(common_on_freq))
print('freq off: ', len(common_off_freq))
print('freq mix: ', len(common_mix_freq))

print('int on: ', len(common_on_int))
print('int off: ', len(common_off_int))
print('int mix: ', len(common_mix_int))
